Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### L2 regularization to Logistic model

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # l2 regularization 
  l2_weights = tf.nn.l2_loss(weights)

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights + l2_weights) + biases



  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.856144
Minibatch accuracy: 10.9%
Validation accuracy: 14.3%
Minibatch loss at step 500: 1.477867
Minibatch accuracy: 78.1%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 0.984594
Minibatch accuracy: 78.9%
Validation accuracy: 76.9%
Minibatch loss at step 1500: 1.061599
Minibatch accuracy: 78.1%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 1.212981
Minibatch accuracy: 73.4%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 0.876879
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 1.246878
Minibatch accuracy: 73.4%
Validation accuracy: 78.7%
Minibatch loss at step 3500: 0.845944
Minibatch accuracy: 75.8%
Validation accuracy: 78.8%
Minibatch loss at step 4000: 0.981809
Minibatch accuracy: 78.1%
Validation accuracy: 79.0%
Minibatch loss at step 4500: 0.908044
Minibatch accuracy: 78.1%
Validation accuracy: 79.4%
Minibatch loss at step 5000: 0.510986
Minibatch accuracy: 85.2%
Validation accura

L2 regularization to logistic regression model has improved in the performance when compare to without it.

#### L2 Regularization to Neural Network Model

In [7]:
batch_size = 128

graph_1 = tf.Graph()
with graph_1.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases_input = tf.Variable(tf.zeros([1024]))
    
  weights_relu = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases_relu = tf.Variable(tf.zeros([num_labels]))
    
  # l2 regularization 
  l2_weights_input = tf.nn.l2_loss(weights_input)
  l2_weights_relu = tf.nn.l2_loss(weights_relu)
    
  # Training computation.
  input_1 = tf.matmul(tf_train_dataset, weights_input) + biases_input

  relu = tf.nn.relu(input_1)
  
  logits = tf.matmul(relu, weights_relu + l2_weights_relu) + biases_relu
    
  #logits = tf.nn.dropout(logits, rate=0.5)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input), weights_relu) + biases_relu)
  test_prediction =  tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input), weights_relu) + biases_relu)

In [8]:
num_steps = 10001

with tf.Session(graph=graph_1) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 416.935303
Minibatch accuracy: 11.7%
Validation accuracy: 38.0%
Minibatch loss at step 500: 23.701281
Minibatch accuracy: 78.1%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 7.165300
Minibatch accuracy: 82.8%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 13.988478
Minibatch accuracy: 71.9%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 3.696696
Minibatch accuracy: 80.5%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 2.455560
Minibatch accuracy: 88.3%
Validation accuracy: 82.4%
Minibatch loss at step 3000: 3.436885
Minibatch accuracy: 78.9%
Validation accuracy: 81.0%
Minibatch loss at step 3500: 4.001436
Minibatch accuracy: 82.0%
Validation accuracy: 81.1%
Minibatch loss at step 4000: 2.400054
Minibatch accuracy: 85.2%
Validation accuracy: 82.7%
Minibatch loss at step 4500: 1.656310
Minibatch accuracy: 78.9%
Validation accuracy: 82.2%
Minibatch loss at step 5000: 2.574886
Minibatch accuracy: 86.7%
Validation acc

* It is observed that l2 weights when added to input - hidden layer weights or rather simple input weights then the model do not performs good. This may indicate that l2 loss should not be applied to all weights in the NN but only at the last layer.
* when l2 weights are added to only last layer it peformed good. The results are better than logistic with l2 regularization and as well as NN without l2 regularization.
* note that here regularization is applied only to training and not testing or validation.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
batch_size = 128

graph_1 = tf.Graph()
with graph_1.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases_input = tf.Variable(tf.zeros([1024]))
    
  weights_relu = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases_relu = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  input_1 = tf.matmul(tf_train_dataset, weights_input) + biases_input

  relu = tf.nn.relu(input_1)
  
  logits = tf.matmul(relu, weights_relu) + biases_relu

   
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input), weights_relu) + biases_relu)
  test_prediction =  tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input), weights_relu) + biases_relu)

In [10]:
num_steps = 3001

with tf.Session(graph=graph_1) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 358.135437
Minibatch accuracy: 10.9%
Validation accuracy: 33.1%
Minibatch loss at step 500: 26.265766
Minibatch accuracy: 77.3%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 5.478290
Minibatch accuracy: 83.6%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 6.567412
Minibatch accuracy: 80.5%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 4.289850
Minibatch accuracy: 77.3%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 1.719467
Minibatch accuracy: 89.8%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 5.444479
Minibatch accuracy: 75.0%
Validation accuracy: 81.8%
Test accuracy: 89.3%


Observation of Test accuracy for different batch sizes: 
* num_steps = 5 ;  Test accuracy: 64.6%
* num_steps = 50 ;  Test accuracy: 83.2%
* num_steps = 500 ;  Test accuracy:  87.7%
* num_steps = 3001 ;  Test accuracy:  88.5%

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
batch_size = 128

graph_1 = tf.Graph()
with graph_1.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases_input = tf.Variable(tf.zeros([1024]))
    
  weights_relu = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases_relu = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  input_1 = tf.matmul(tf_train_dataset, weights_input) + biases_input

  relu = tf.nn.relu(input_1)
  
  logits = tf.matmul(relu, weights_relu) + biases_relu
    
  #dropouts
  logits_drop = tf.nn.dropout(logits,rate=0.25)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_drop))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input), weights_relu) + biases_relu)
  test_prediction =  tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input), weights_relu) + biases_relu)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
num_steps = 5

with tf.Session(graph=graph_1) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 368.560089
Minibatch accuracy: 12.5%
Validation accuracy: 18.2%
Test accuracy: 61.4%


Observation of Test accuracy for different batch sizes: 
* num_steps = 5 ;  Test accuracy: 62.7%
* num_steps = 50 ;  Test accuracy: 85.2%
* num_steps = 500 ;  Test accuracy:  88.0%
* num_steps = 3001 ;  Test accuracy: 88.3 %

Dropouts converges to a good result with small batch size

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [13]:
batch_size = 128

graph_1 = tf.Graph()
with graph_1.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, 2048]))
  biases_input = tf.Variable(tf.zeros([2048]))
    
  weights_relu_1 = tf.Variable(
    tf.truncated_normal([2048, 1024]))
  biases_relu_1 = tf.Variable(tf.zeros([1024]))
    
  weights_relu_2 = tf.Variable(
    tf.truncated_normal([1024, 512]))
  biases_relu_2 = tf.Variable(tf.zeros([512]))
    
  weights_relu_3 = tf.Variable(
    tf.truncated_normal([512, num_labels]))
  biases_relu_3 = tf.Variable(tf.zeros([num_labels]))
    
 
  # Training computation.
  input_1 = tf.matmul(tf_train_dataset, weights_input) + biases_input

  relu_1 = tf.nn.dropout( tf.nn.relu(input_1), rate=0.1)
    
  logits_1 = tf.matmul(relu_1, weights_relu_1) + biases_relu_1
    
  relu_2 = tf.nn.dropout(tf.nn.relu(logits_1), rate=0.1) 

  logits_2 = tf.matmul(relu_2, weights_relu_2) + biases_relu_2

  relu_3 = tf.nn.dropout(tf.nn.relu(logits_2), rate=0.1)
    
  logits = tf.matmul(relu_3, weights_relu_3) + biases_relu_3

    
  #dropouts
  #logits_drop = tf.nn.dropout(logits,rate=0.25)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)


  tf_valid = tf.matmul(tf_valid_dataset, weights_input) + biases_input
  tf_valid = tf.nn.relu(tf_valid)
  tf_valid = tf.matmul(tf_valid, weights_relu_1) + biases_relu_1
  tf_valid = tf.nn.relu(tf_valid)
  tf_valid = tf.matmul(tf_valid, weights_relu_2) + biases_relu_2
  tf_valid = tf.nn.relu(tf_valid)
  tf_valid = tf.matmul(tf_valid, weights_relu_3) + biases_relu_3

  valid_prediction = tf.nn.softmax(tf_valid)

  tf_test = tf.matmul(tf_test_dataset, weights_input) + biases_input
  tf_test = tf.nn.relu(tf_test)
  tf_test = tf.matmul(tf_test, weights_relu_1) + biases_relu_1
  tf_test = tf.nn.relu(tf_test)
  tf_test = tf.matmul(tf_test, weights_relu_2) + biases_relu_2
  tf_test = tf.nn.relu(tf_test)
  tf_test = tf.matmul(tf_test, weights_relu_3) + biases_relu_3

  test_prediction =  tf.nn.softmax(tf_test)

In [14]:
num_steps = 50

with tf.Session(graph=graph_1) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 155330.312500
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Test accuracy: 10.0%


### Tensorflow Keras layer usage for the problem

In [16]:
#Load data
pickle_file = 'notMNIST.pickle'
batch_size = 128

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [17]:
def myGenerator():
    for batch in np.random.choice(train_dataset.shape[0]
                     ,size=(train_dataset.shape[0]//batch_size, batch_size)
                     ,replace=False
                    ):
        yield train_dataset[batch, :], train_labels[batch]
        

In [28]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(2048, activation=tf.nn.relu),
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(32, activation=tf.nn.relu),
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
#model.fit(train_dataset[:5000,:], train_labels[:5000], epochs=5) #testing the with small sample
model.fit_generator(myGenerator()
                    , steps_per_epoch=train_dataset.shape[0]//batch_size//5
                    , epochs=5
                    , validation_data=(valid_dataset,valid_labels)
                   )

Epoch 1/5
312/312 [==============================] - 31s 98ms/step - loss: 0.7165 - acc: 0.7859 - val_loss: 0.5229 - val_acc: 0.8379
Epoch 2/5
312/312 [==============================] - 30s 95ms/step - loss: 0.5563 - acc: 0.8349 - val_loss: 0.5103 - val_acc: 0.8455
Epoch 3/5
312/312 [==============================] - 31s 101ms/step - loss: 0.5352 - acc: 0.8415 - val_loss: 0.4587 - val_acc: 0.8590
Epoch 4/5
312/312 [==============================] - 28s 91ms/step - loss: 0.4933 - acc: 0.8534 - val_loss: 0.4709 - val_acc: 0.8564
Epoch 5/5
312/312 [==============================] - 28s 90ms/step - loss: 0.4786 - acc: 0.8601 - val_loss: 0.4451 - val_acc: 0.8648


In [30]:
model.evaluate(test_dataset, test_labels)

10000/10000 [==============================] - 2s 236us/sample - loss: 0.2359 - acc: 0.9337


[0.23588024379760028, 0.9337]